# Testing with a dataset 

We compare different model on the dataset of wikiann that contains tags `['PER', 'ORG', 'LOC']`
We get teh score by comparing the output at the entity level and not at the token level.

In [9]:
import pickle
from collections import defaultdict
from ner.process_results import get_metrics, get_metrics_all, show_cm, show_cm_multi, get_metrics_all_extended, show_cm_multi_extended
from ner.classical_models.ner_models import get_results
from ner.Datasets.Conll2003Dataset import Conll2003Dataset
from ner.Datasets.OntoNotes5Dataset import OntoNote5Dataset
from ner.Datasets.MyDataset import MyDataset

from ner.utils import *

%load_ext autoreload
%autoreload 2

# dataset = MyDataset.my_load_dataset(dataset=Conll2003Dataset, split = 'test', cleaned= True)

## Getting results with Ontonote dataset

In [10]:
train, test = OntoNote5Dataset.my_load_dataset()

Filter:   0%|          | 0/9873 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1403 [00:00<?, ? examples/s]

In [11]:
results, f1s= get_results(test, with_save= True)
f1s

 13%|█▎        | 179/1402 [11:29<1:13:40,  3.61s/it]

# Others

In [ ]:
file_name = results_files[10]
file_path = folder_path + file_name
with open(file_path, 'rb') as f:
    results_llama[file_name]= pickle.load(f) 
print(results_llama[file_name].keys()) 
results_llama[file_name]['true_labels'] = dataset_llama['spans']
cm = {}
f1 = {}
precision = {}
recall = {}
y_true, y_pred, all_nes= {},{}, {}
for model in results_llama[file_name] : 
    if model != "true_labels" :
        cm[model],f1[model], precision[model], recall[model], y_true[model], y_pred[model], all_nes[model]= get_metrics_all(results_llama[file_name][model], results_llama[file_name]['true_labels'])
        show_cm_multi(cm[model],f1[model], precision[model], recall[model], model)
print(f"{file_name} {f1}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from ner.classical_models.ner_models import *

tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang Mozart and I live in Berlin"

ner_results = nlp(example)
print(ner_results)
print(from_bert_to_tags(ner_results, "distilbert"))

[{'entity': 'LABEL_0', 'score': 0.9992781, 'index': 1, 'word': 'My', 'start': 0, 'end': 2}, {'entity': 'LABEL_0', 'score': 0.9994466, 'index': 2, 'word': 'name', 'start': 3, 'end': 7}, {'entity': 'LABEL_0', 'score': 0.99946386, 'index': 3, 'word': 'is', 'start': 8, 'end': 10}, {'entity': 'LABEL_1', 'score': 0.9927504, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'LABEL_2', 'score': 0.9861588, 'index': 5, 'word': 'Mozart', 'start': 20, 'end': 26}, {'entity': 'LABEL_0', 'score': 0.9996026, 'index': 6, 'word': 'and', 'start': 27, 'end': 30}, {'entity': 'LABEL_0', 'score': 0.99673045, 'index': 7, 'word': 'I', 'start': 31, 'end': 32}, {'entity': 'LABEL_0', 'score': 0.99919397, 'index': 8, 'word': 'live', 'start': 33, 'end': 37}, {'entity': 'LABEL_0', 'score': 0.9991404, 'index': 9, 'word': 'in', 'start': 38, 'end': 40}, {'entity': 'LABEL_5', 'score': 0.9969441, 'index': 10, 'word': 'Berlin', 'start': 41, 'end': 47}]


In [7]:
import spacy
from spacy.lang.en.examples import sentences 

nlp = spacy.load("en_core_web_trf")
doc = nlp(sentences[0])
print(doc.ents)
entities = [(ent.text, ent.label_, ent.label_) for ent in doc.ents]
entities

(Apple, U.K., $1 billion)


[('Apple', 'ORG', 'ORG'),
 ('U.K.', 'GPE', 'GPE'),
 ('$1 billion', 'MONEY', 'MONEY')]

In [ ]:
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")

model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")

classifier = pipeline("ner", model=model, tokenizer=tokenizer)

results = classifier("Alya told Jasmine that Andrew could pay with cash..")

print(results)
print(from_bert_to_tags(results, "roberta"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER',
  'score': 0.9997861,
  'index': 1,
  'word': '▁Al',
  'start': 0,
  'end': 2},
 {'entity': 'I-PER',
  'score': 0.9998591,
  'index': 2,
  'word': 'ya',
  'start': 2,
  'end': 4},
 {'entity': 'I-PER',
  'score': 0.99995816,
  'index': 4,
  'word': '▁Jasmin',
  'start': 10,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.9999584,
  'index': 5,
  'word': 'e',
  'start': 16,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.99998057,
  'index': 7,
  'word': '▁Andrew',
  'start': 23,
  'end': 29}]

In [ ]:

cm = {}
f1 = {}
precision = {}
recall = {}
y_true, y_pred, all_nes= {},{}, {}
res = results
for model in res: 
    if model != "true_labels" :
        cm[model],f1[model], precision[model], recall[model], y_true[model], y_pred[model], all_nes[model]= get_metrics_all(res[model], res['true_labels'])
        # show_cm(cm[model], [{tag : f"{f1s[model][tag]:.2f}" } for tag in ["general"] + tag_type], model)
        show_cm_multi(cm[model],f1[model], precision[model], recall[model], model)


In [ ]:
cm = {}
f1 = {}
precision = {}
recall = {}
y_true, y_pred, all_nes= {},{}, {}
res = results
for model in res : 
    if model in ['spacy', 'refined'] :
        cm[model],f1[model], precision[model], recall[model], y_true[model], y_pred[model], all_nes[model]= get_metrics_all_extended(res[model], res['true_labels'])
        # show_cm(cm[model], [{tag : f"{f1s[model][tag]:.2f}" } for tag in ["general"] + tag_type], model)
        show_cm_multi_extended(cm[model],f1[model], precision[model], recall[model], model)

In [ ]:
import pandas as pd 
key ="entity"
resu = pd.DataFrame({"gold" : y_true[key], "pred" : y_pred[key], 'named_entity' : all_nes[key]})
resu[resu['gold'] != resu['pred']]

In [ ]:
dfs = {}
for key in results :
    flattened = [item for sublist in results[key] for item in sublist]
    dfs[key] = pd.DataFrame([{'entity' : ne[0], 'tag' : ne[1], 'ref_tag' : ne[2] if len(ne)>2 else ""} for ne in flattened])
    dfs[key] = dfs[key].groupby('entity').agg(set)
dfs['true_labels'].rename(columns={'tag': 'gold_tags'}, inplace=True)
del dfs['true_labels']['ref_tag']

for key in results :
    if key != "true_labels" :
        dfs[key] = dfs['true_labels'].merge(dfs[key], on='entity', how='outer')
        dfs[key].fillna(-1, inplace=True)


for key in results_llama :
    flattened = [item for sublist in results_llama[key] for item in sublist]
    dfs[key] = pd.DataFrame([{'entity' : ne, 'tag' : tag} for ne, tag in flattened])
    dfs[key] = dfs[key].groupby('entity').agg(set)
dfs['true_labels'].rename(columns={'tag': 'gold_tags'}, inplace=True)
for key in results_llama :
    if key != "true_labels" :
        dfs[key] = dfs['true_labels'].merge(dfs[key], on='entity', how='outer')
        dfs[key].fillna(-1, inplace=True)

In [ ]:
key = 'few_shots_random'
tag_distr_missed_by_model(key), tag_distr_added_by_model(key)

In [ ]:
key = 'few_shots_sentence'
tag_distr_missed_by_model(key), tag_distr_added_by_model(key)

In [ ]:
key = 'entity'
tag_distr_missed_by_model(key), tag_distr_added_by_model(key)

In [ ]:
key = 'refined'
tag_distr_missed_by_model(key), tag_distr_added_by_model(key)
# temp = dfs[key][dfs[key]['gold_tags'] != -1]
# temp[temp['tag'] == {'MISC'}][['gold_tags','tag_precision_y']]['tag_precision_y'].value_counts()